# 🛍️ | Cora-For-Zava: Collect Agent Spans Locally

Welcome! This notebook teaches you how to capture spans produced by agent workflows for local validation before exporting to Azure Monitor.

## 🛒 Our Zava Scenario

**Cora** is a customer service chatbot for **Zava** - a fictitious retailer of home improvement goods for DIY enthusiasts. Before sending telemetry data to production monitoring systems, it's important to validate the spans locally. This notebook shows you how to configure an in-memory exporter to capture and inspect agent traces, ensuring your telemetry is correct before deploying to Azure Monitor or other backends.

## 🎯 What You'll Build

By the end of this notebook, you'll have:
- ✅ Configured an in-memory span exporter for local testing
- ✅ Recorded synthetic agent interactions with proper spans
- ✅ Inspected captured spans to validate telemetry structure
- ✅ Learned best practices for local telemetry validation

## 💡 What You'll Learn

- How to configure an in-memory exporter for testing
- How to capture spans without sending to external systems
- How to inspect and validate span structure locally
- Best practices for telemetry testing before production deployment

Ready to capture and validate spans? Let's get started! 🚀

---

## 1. Configure an in-memory exporter
This setup mirrors what you would ship in production, but keeps traces in memory for quick inspection. Add an OTLP exporter alongside it when you are ready to emit to a collector.

In [ ]:
from opentelemetry import trace
from opentelemetry.sdk.resources import Resource
from opentelemetry.sdk.trace import TracerProvider
from opentelemetry.sdk.trace.export import SimpleSpanProcessor
from opentelemetry.sdk.trace.export.in_memory_span_exporter import InMemorySpanExporter
from opentelemetry.trace import SpanKind
import json

resource = Resource.create({"service.name": "cora-agent-demo", "service.namespace": "ignite25"})
provider = TracerProvider(resource=resource)
trace.set_tracer_provider(provider)
memory_exporter = InMemorySpanExporter()
provider.add_span_processor(SimpleSpanProcessor(memory_exporter))

tracer = trace.get_tracer("labs.5.observability.snapshot")
print("Tracer initialised with in-memory exporter.")

## 2. Record a synthetic agent interaction
The helper below reuses the same attribute set you saw in the previous notebook.

In [ ]:
def record_agent_interaction():
    system_prompt = (
        "You are Cora, a polite, factual, and helpful Zava retail assistant. "
        "Answer with concise, markdown-friendly responses."
    )
    customer_prompt = "Do you have a satin finish paint that dries fast?"
    agent_reply = (
        "⚡ Absolutely! Zava SwiftCoat Satin at $32 dries in 30 minutes. "
        "Want rollers too?"
    )
    conversation_id = "retail-session-002"
    tool_call_id = "call-17"
    tool_arguments = {"product_type": "paint", "finish": "satin"}
    tool_result = {
        "sku": "PAINT-FAST-221",
        "name": "Zava SwiftCoat Satin",
        "inventory": 142
    }
    tool_definitions = [
        {
            "type": "function",
            "name": "inventory_lookup",
            "description": "Fetches inventory details for a specific product category and finish.",
            "parameters": {
                "type": "object",
                "properties": {
                    "product_type": {
                        "type": "string",
                        "description": "Product category name to search."
                    },
                    "finish": {
                        "type": "string",
                        "description": "Desired paint finish (e.g., satin, matte)."
                    }
                },
                "required": ["product_type"]
            }
        }
    ]
    input_messages = [
        {
            "role": "system",
            "parts": [
                {"type": "text", "content": system_prompt}
            ],
        },
        {
            "role": "user",
            "parts": [
                {"type": "text", "content": customer_prompt}
            ],
        },
        {
            "role": "assistant",
            "parts": [
                {
                    "type": "tool_call",
                    "id": tool_call_id,
                    "name": "inventory_lookup",
                    "arguments": tool_arguments
                }
            ],
        },
        {
            "role": "tool",
            "parts": [
                {
                    "type": "tool_call_response",
                    "id": tool_call_id,
                    "result": tool_result
                }
            ],
        },
    ]
    output_messages = [
        {
            "role": "assistant",
            "parts": [
                {"type": "text", "content": agent_reply}
            ],
            "finish_reason": "stop"
        }
    ]

    with tracer.start_as_current_span("invoke_agent cora-retail-agent", kind=SpanKind.CLIENT) as span:
        span.set_attribute("gen_ai.provider.name", "azure.ai.inference")
        span.set_attribute("gen_ai.operation.name", "invoke_agent")
        span.set_attribute("gen_ai.agent.name", "cora-retail-agent")
        span.set_attribute("gen_ai.agent.id", "agents/cora-retail-agent")
        span.set_attribute("gen_ai.agent.description", "Cora retail assistant for Zava DIY customers.")
        span.set_attribute("gen_ai.request.model", "gpt-4o-mini")
        span.set_attribute("gen_ai.request.max_tokens", 256)
        span.set_attribute("gen_ai.request.temperature", 0.3)
        span.set_attribute("gen_ai.request.top_p", 0.9)
        span.set_attribute("gen_ai.response.model", "gpt-4o-mini")
        span.set_attribute("gen_ai.response.finish_reasons", ["stop"])
        span.set_attribute("gen_ai.response.id", "resp-31f1")
        span.set_attribute("gen_ai.usage.input_tokens", 98)
        span.set_attribute("gen_ai.usage.output_tokens", 41)
        span.set_attribute("gen_ai.conversation.id", conversation_id)
        span.set_attribute("gen_ai.system_instructions", system_prompt)
        span.set_attribute("gen_ai.tool.definitions", json.dumps(tool_definitions, ensure_ascii=False))
        span.set_attribute("gen_ai.input.messages", json.dumps(input_messages, ensure_ascii=False))
        span.set_attribute("gen_ai.output.messages", json.dumps(output_messages, ensure_ascii=False))
        span.set_attribute("server.address", "cora-agents.eastus2.inference.ai.azure.com")
        span.set_attribute("server.port", 443)

        with tracer.start_as_current_span("execute_tool inventory_lookup", kind=SpanKind.INTERNAL) as tool_span:
            tool_span.set_attribute("gen_ai.operation.name", "execute_tool")
            tool_span.set_attribute("gen_ai.tool.name", "inventory_lookup")
            tool_span.set_attribute("gen_ai.tool.type", "function")
            tool_span.set_attribute("gen_ai.tool.call.id", tool_call_id)
            tool_span.set_attribute("gen_ai.tool.call.arguments", json.dumps(tool_arguments))
            tool_span.set_attribute("gen_ai.tool.call.result", json.dumps(tool_result))

record_agent_interaction()
print("Interaction recorded.")

## 3. Inspect the captured spans
The span objects expose attributes, timing, and resource metadata. Convert them into dictionaries to confirm the payload matches the spec before shipping to a collector.

In [ ]:
captured = memory_exporter.get_finished_spans()
print(f"Captured {len(captured)} spans")

def span_to_dict(span):
    return {
        "name": span.name,
        "context": {"span_id": span.context.span_id, "trace_id": span.context.trace_id},
        "kind": span.kind.name,
        "attributes": dict(span.attributes),
        "resource": dict(span.resource.attributes),
        "status": span.status.status_code.name
    }

span_snapshots = [span_to_dict(span) for span in captured]
for snapshot in span_snapshots:
    print(json.dumps(snapshot, indent=2, ensure_ascii=False))

# Reset exporter so repeated notebook runs do not duplicate results
memory_exporter.clear()
print("Exporter cleared.")

### Optional: Wire an OTLP exporter
When you are ready to integrate with Azure Monitor, add an OTLP exporter (gRPC or HTTP) alongside the in-memory exporter.

```python
from opentelemetry.exporter.otlp.proto.http.trace_exporter import OTLPSpanExporter
provider.add_span_processor(BatchSpanProcessor(
    OTLPSpanExporter(
        endpoint="https://<region>.monitor.azure.com/v2/track",
        headers={"Authorization": "Bearer <token>"}
    )
)
))
```

Replace the endpoint and headers with the values provided by your Azure Monitor workspace.